In [1]:
import pandas as pd
import psycopg2

In [2]:
# Параметры для подключения к БД

In [3]:
#  Создаем соединение с БД
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

**Задание 3.1**

Сколько вакансий есть в базе?

In [4]:
# код запроса представляет из себя строковую переменную
# n = 10
query_3_1 = f'''select * 
                from public.VACANCIES 
             '''
# используем f-строки - так удобнее передавать параметры

df_vacancies = pd.read_sql_query(query_3_1, connection)

In [5]:
print('Сколько вакансий есть в базе?: {}'.format(df_vacancies.shape[0]))

Сколько вакансий есть в базе?: 49197


**Задание 3.2**

Сколько работодателей?

In [6]:
query_3_2 = f'''select * 
            from public.EMPLOYERS 
         '''

df_employers = pd.read_sql_query(query_3_2, connection)

In [7]:
print('Сколько работодателей?: {}'.format(df_employers.shape[0]))

Сколько работодателей?: 23501


**Задание 3.3**

Сколько регионов?

In [8]:
query_3_3 = f'''select * 
            from public.AREAS 
         '''

df_areas = pd.read_sql_query(query_3_3, connection)

In [9]:
print('Сколько регионов?: {}'.format(df_areas.shape[0]))

Сколько регионов?: 1362


**Задание 3.4**

Сколько сфер деятельности?

In [10]:
query_3_4 = f'''select * 
            from public.INDUSTRIES 
         '''

df_industries = pd.read_sql_query(query_3_4, connection)

In [11]:
print('Сколько сфер деятельности?: {}'.format(df_industries.shape[0]))

Сколько сфер деятельности?: 294


**дание 4.1**

Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).

In [12]:
query_4_1 = f'''SELECT 
                areas.name, 
                count (vacancies.id) as cnt
            FROM
                public.vacancies,
                public.areas 
            WHERE vacancies.area_id = areas.id 
            group by areas.name
            order by cnt desc
            limit 10
         '''            

df = pd.read_sql_query(query_4_1, connection)
df

,name,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
5,Екатеринбург,1698
6,Нижний Новгород,1670
7,Казань,1415
8,Краснодар,1301
9,Самара,1144


**Вывод:** видим кол-во вакансий зависит от численности города.

**Задание 4.2**

Посмотрим на зарплаты. У какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой?

In [13]:
query_4_2 = f'''SELECT 
                count (vacancies.id) as cnt
            FROM
                public.vacancies
            WHERE salary_to != 0 or salary_from !=0
         '''  

df = pd.read_sql_query(query_4_2, connection)
df

,cnt
0,24073


**Задание 4.3**

Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого числа.

In [14]:
query_4_3 = f'''SELECT 
                AVG(salary_to) as top,
                AVG(salary_from) as lower
            FROM
                public.vacancies
         ''' 
df = pd.read_sql_query(query_4_3, connection)

In [15]:
print('Средние значения для нижней {} и верхней границы зарплатной вилки: {}'.
      format(round(df.at[0,'lower'],0), round(df.at[0,'top'],0)))

Средние значения для нижней 71065.0 и верхней границы зарплатной вилки: 110537.0


**Вывод:** среднее значение верхней и нижней границы имеет небольшую вилку, в отличие от верхней и нижней границы отдельно взятой вакансии. 

**Задание 4.4**

Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Какая пара находится на втором месте по популярности?

In [16]:
query_4_4 = f''' SELECT 
                  schedule, 
                  employment, 
                  COUNT(*) AS count /*подсчёт всех строк*/
              FROM public.vacancies /*из таблицы public.vacancies*/
              GROUP BY 1, 2 /*группировка по столбцам 1 и 2*/
              ORDER BY count DESC /*сортировка по столбцу кол-во*/
              limit 2
          '''    

df = pd.read_sql_query(query_4_4, connection)
df

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802


**Задание 4.5**

Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.
Выберите верный вариант ответа.

In [17]:
query_4_5 = f''' SELECT 
                  experience, 
                  COUNT(*) AS count /*подсчёт всех строк*/
              FROM public.vacancies /*из таблицы public.vacancies*/
              GROUP BY 1/*группировка по столбцу 1*/
              ORDER BY count /*сортировка по столбцу кол-во*/
              limit 4
          '''    

df = pd.read_sql_query(query_4_5, connection)
df

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


**Вывод:** колличестов вакансий уменьшается с увеличением требования к опыту работы

**Задание 5.1**

Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.
Выберите верный вариант ответа:

In [18]:
query_5_1 = f'''select e.name, 
                count(e.name) count_employers
            from public.employers e
                left join public.vacancies v on v.employer_id = e.id
            group by e.name
            order by count_employers desc
            limit 5
          '''    
df = pd.read_sql_query(query_5_1, connection)
df

,name,count_employers
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


**Вывод:** молодые IT компании требуют больше рекрутеров в свой штат.

**Задание 5.2**

Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.

Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей. Впишите его название в поле ниже в том виде, который вернул запрос.

In [19]:
query_5_2 = f'''SELECT
                    a.name area,
                    COUNT(distinct e.id) empcnt,
                    COUNT(distinct v.id) vaccnt
                FROM public.areas a
                    left JOIN public.employers e ON a.id = e.area
                    left JOIN public.vacancies v ON a.id = v.area_id
                WHERE v.name IS NULL
                GROUP BY 1
                ORDER BY 2 DESC
                limit 5
              '''
df = pd.read_sql_query(query_5_2, connection)
df 

,area,empcnt,vaccnt
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0


**Задание 5.3**

Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии.

Выберите максимальное значение из получившегося списка.

In [20]:
query_5_3 = f'''SELECT
                    e.name "Работодатель",
                    COUNT(distinct a.name) name_cnt
                FROM public.vacancies v
                    left JOIN public.employers e ON v.employer_id = e.id
                    left JOIN public.areas a ON v.area_id = a.id
                GROUP BY 1
                ORDER BY 2 DESC
                limit 5
              '''
df = pd.read_sql_query(query_5_3, connection)
df 

,Работодатель,name_cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


**Вывод:** чем разнообразнее спектр услуг IT компании, тем она болше представлена в различных регионах

**Задание 5.4**

Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

Введите количество, которое вернул запрос, в поле ниже.

In [21]:
query_5_4 = f'''SELECT
                    count(e.name) cnt
                FROM
                    public.EMPLOYERS e
                    LEFT JOIN public.EMPLOYERS_INDUSTRIES ei on employer_id = e.id
                where ei.industry_id is NULL
                limit 5
             '''   
df = pd.read_sql_query(query_5_4, connection)
df 

,cnt
0,8419


**Задание 5.5**

Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

Введите в поле ниже название этой компании так же, как оно указано в результате запроса.

In [22]:
query_5_5 = f'''SELECT
                    e.name,
                    count (i.name) cnt
                FROM
                    public.EMPLOYERS_INDUSTRIES ei
                    JOIN public.EMPLOYERS e on ei.employer_id = e.id
                    JOIN public.INDUSTRIES i on ei.industry_id = i.id
                        
                GROUP BY 1
                    having count (i.name) = 4
                ORDER BY 1 
                limit 3
             '''   
df = pd.read_sql_query(query_5_5, connection)
df                 

,name,cnt
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4


**Задание 5.6**

С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».

In [23]:
query_5_6 = f'''SELECT
                    count (e.id) cnt
                FROM    
                    public.EMPLOYERS e
                    JOIN public.EMPLOYERS_INDUSTRIES ei on e.id = ei.employer_id
                    JOIN public.INDUSTRIES i on ei.industry_id = i.id
                where (i.name) = 'Разработка программного обеспечения'        
             '''
df = pd.read_sql_query(query_5_6, connection)
df  

,cnt
0,3553


**Задание 5.7**

Для компании «Яндекс» выведите список регионов-миллионников , в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Должна получиться выборка такого вида (приведён пример результата для компании SberTech):

In [24]:
query_5_7 = f'''SELECT
                    a.name,
                    COUNT(v.id) AS cnt
                FROM public.vacancies AS v 
                JOIN public.employers AS e ON e.id = v.employer_id AND e.name = 'Яндекс'
                JOIN public.areas AS a ON a.id = v.area_id AND
                    a.name IN ('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань',
                               'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 
                               'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')
                GROUP BY a.name
                UNION ALL
                SELECT
                    'Total',
                    COUNT(*)
                FROM public.vacancies AS vv
                JOIN public.employers AS ee ON ee.id = vv.employer_id AND ee.name = 'Яндекс'
                JOIN public.areas AS aa ON aa.id = vv.area_id AND
                    aa.name IN ('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань',
                               'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 
                               'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')
                ORDER BY cnt                                    
             '''
df = pd.read_sql_query(query_5_7, connection)
df  

,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


**Вывод:** видим кол-во вакансий увеличивается с численностью города

**Задание 6.1**

Сколько вакансий имеет отношение к данным?

In [25]:
query_6_1 = f'''SELECT
                    count (*) cnt
                FROM    
                    public.VACANCIES v
                where  LOWER (v.name) LIKE '%data%'
                    OR LOWER (v.name) LIKE '%данн%'
             '''
df = pd.read_sql_query(query_6_1, connection)
df  

,cnt
0,1771


**Задание 6.2**

Сколько есть подходящих вакансий для начинающего дата-сайентиста?

Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:

* ‘data scientist’;
* ‘data science’;
* ‘исследователь данных’;
* ‘ML’ (здесь не нужно брать вакансии по HTML);
* ‘machine learning’;
* ‘машинн%обучен%’.
>В следующих заданиях мы продолжим работать с вакансиями по указанному выше условию (уже не учитывая вакансии уровня *Junior*).

Считаем вакансиями для специалистов уровня *Junior* следующие:

в названии есть слово “junior” **или**        
требуемый опыт — «Нет опыта» **или**          
тип трудоустройства — «Стажировка».

In [26]:
query_6_2 = f'''SELECT
                    count(*)
                FROM vacancies v
                
                WHERE (LOWER (v.name) LIKE '%data scientist%' 
                    OR LOWER (v.name) LIKE '%data science%' 
                    OR LOWER (v.name) LIKE '%исследователь данных%' 
                    OR (
                       UPPER (v.name) LIKE '%ML%' 
                       AND UPPER (v.name) NOT LIKE '%HTML%'
                       ) 
                    OR LOWER (v.name) LIKE '%machine learning%' 
                    OR LOWER (v.name) LIKE '%машинн%обучен%')
                    AND (
                        LOWER (v.name) LIKE '%junior%'
                        OR LOWER (v.experience) LIKE '%нет опыта%'
                        OR LOWER (v.employment) LIKE '%стажировка%'
                        )
                 '''   
df = pd.read_sql_query(query_6_2, connection)
df      

,count
0,51


**Вывод:** начинающему специалисту тяжело найти подходящую вакансию.

**Задание 6.3**

Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

В качестве ответа запишите количество, которое вернул запрос.

In [27]:
query_6_3 = f'''SELECT
                    count(*)
                from vacancies v  /*из таблицы вакансий*/
                WHERE (LOWER (v.name) LIKE '%data scientist%' 
                    OR LOWER (v.name) LIKE '%data science%' 
                    OR LOWER (v.name) LIKE '%исследователь данных%' 
                    OR ( 
                       v.name LIKE '%ML%' 
                       AND v.name NOT LIKE '%HTML%'
                       ) 
                    OR LOWER (v.name) LIKE '%machine learning%' 
                    OR LOWER (v.name) LIKE '%машинн%обучен%')
                    AND (UPPER (v.key_skills) LIKE '%SQL%' or LOWER (v.key_skills) LIKE '%postgres%')
            '''
df = pd.read_sql_query(query_6_3, connection)
df   

,count
0,201


**Задание 6.4**

С помощью запроса, аналогичного предыдущему, проверьте, насколько популярен Python в требованиях работодателей к DS. Вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

В качестве ответа запишите количество, которое вернул запрос.

In [28]:
query_6_4 = f'''SELECT
                    count(*)
                from vacancies v  /*из таблицы вакансий*/
                WHERE (LOWER (v.name) LIKE '%data scientist%' 
                    OR LOWER (v.name) LIKE '%data science%' 
                    OR LOWER (v.name) LIKE '%исследователь данных%' 
                    OR ( 
                       v.name LIKE '%ML%' 
                       AND v.name NOT LIKE '%HTML%'
                       ) 
                    OR LOWER (v.name) LIKE '%machine learning%' 
                    OR LOWER (v.name) LIKE '%машинн%обучен%')
                    AND LOWER (v.key_skills) LIKE '%python%'
            '''
df = pd.read_sql_query(query_6_4, connection)
df 

,count
0,351


**Вывод:** видим что спецалист со знанием Python более востребован чем специалист со знанием SQL или postgres.

**Задание 6.5**

Сколько ключевых навыков в среднем указывают в вакансиях для DS?

Ответ округлите до двух знаков после точки-разделителя.

In [29]:
query_6_5 = f'''SELECT
                to_char(
                AVG(
                    LENGTH(v.key_skills) - LENGTH(replace(v.key_skills, CHR(9), '')) +1
                    ), '99.99')

                FROM
                    public.VACANCIES v  /*из таблицы вакансий*/
                where (LOWER(v.name) LIKE '%data scientist%'
                OR LOWER (v.name) LIKE '%data science%'
                or LOWER (v.name) LIKE '%исследователь данных%'
                or (v.name LIKE '%ML%' and v.name not LIKE '%HTML%')
                or LOWER (v.name) LIKE '%machine learning%'
                or LOWER (v.name) LIKE '%машинн%обучен%%')
                and v.key_skills is not NULL
             '''   
df = pd.read_sql_query(query_6_5, connection)
df 

,to_char
0,6.41


**Вывод:** из данного запроса видно, что в основном требуются универсальные специалисты

**Задание 6.6**

Напишите запрос, позволяющий вычислить, какую зарплату для DS в среднем указывают для каждого типа требуемого опыта (уникальное значение из поля experience).

При решении задачи примите во внимание следующее:

1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, его и считаем зарплатой по вакансии.
3. Если в расчётах участвует `null`, в результате он тоже даст `null` (посмотрите, что возвращает запрос `select 1 + null`).          
Чтобы избежать этой ситуации, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull) , которая заменит `null` на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0).`                             
Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа.

In [30]:
query_6_6 = f'''SELECT
                    experience,
                    to_char(
                    AVG(COALESCE((salary_from+salary_to)/2, salary_from, salary_to, 0)), '9999999')
                FROM public.vacancies
                WHERE (lower(name) LIKE '%data scientist%'
                    OR lower(name) LIKE '%data science%'
                    OR lower(name) LIKE '%исследователь данных%'
                    OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%')
                    OR lower(name) LIKE '%machine learning%'
                    OR lower(name) LIKE '%машинн%обучен%%')
                    AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
                GROUP BY experience  
             '''   
df = pd.read_sql_query(query_6_6, connection)
df 

,experience,to_char
0,Нет опыта,74643
1,От 1 года до 3 лет,139675
2,От 3 до 6 лет,243115


**Вывод:** видно чем у специалиста больше опыт, тем предлагаетсе выше оплата труда.

#### Дополнительные исследования данных.

**Задание 7.0**          
Узнать среднюю предлагаемую зарплату по региону.

In [31]:
query_7_0 = f'''SELECT
                    a.name "Регион",
                    COUNT(v.id) "Количество вакансий",
                    AVG(COALESCE((salary_from+salary_to)/2, salary_from, salary_to, 0)) "Salary"                    
                FROM public.vacancies v
                    left JOIN public.areas a ON v.area_id = a.id                
                GROUP BY 1
                ORDER BY 3 DESC 
                limit 10
              '''
df = pd.read_sql_query(query_7_0, connection)
df 

,Регион,Количество вакансий,Salary
0,Петропавловка (Республика Бурятия),1,250000.0
1,Дзержинск (Беларусь),3,220349.0
2,Великобритания,2,208962.0
3,Шушары,1,200000.0
4,Заполярный (Ямало-Ненецкий АО),1,200000.0
5,Павловск (Ленинградская область),1,200000.0
6,Нидерланды,2,168750.0
7,Кутаиси,4,165356.0
8,Амурск,1,165000.0
9,Ленск,1,155000.0


**Вывод:** Видно что самые большие предложения по средней зарплате в Бурятии, Беларуси и Великобритании, но на этих данных не правильно делать вывод о средней предлагаемой зарплате по региону, давайте в слеующем запросе отсечем реионы где кол-во предложений меньше 10. 

**Задание 7.1**

Узнаем среднюю предлагаемую зарплату по региону, где кол-во предложений больше 10.

In [32]:
query_7_1 = f'''SELECT
                    a.name "Регион",
                    COUNT(v.id) "Кол-во вакансий",
                    AVG(COALESCE((salary_from+salary_to)/2, salary_from, salary_to, 0)) "Salary"                    
                FROM public.vacancies v
                    left JOIN public.areas a ON v.area_id = a.id                
                GROUP BY 1
                HAVING COUNT(v.id) > 10
                ORDER BY 3 DESC 
                limit 10
              '''
df = pd.read_sql_query(query_7_1, connection)
df 

,Регион,Кол-во вакансий,Salary
0,Турция,39,114699.307692
1,Армения,223,104369.466368
2,Сербия,26,96195.153846
3,США,12,95140.500000
4,Киев,14,92473.785714
5,Магадан,25,88944.880000
6,Баку,91,87286.780220
7,Лобня,16,83281.250000
8,Черногория,14,82454.071429
9,Болгария,12,81087.000000


**Вывод:** Видим, что предложение средней зарплаты выше в зарубежных регионах

In [33]:
# не забываем закрыть соединение после окончания работы
connection.close()